In [279]:
import cobra
import copy
import pandas as pd
import pylab as plt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from scipy import stats
from numpy import array,empty
import six
from sympy import Add, Mul
from sympy.functions.elementary.miscellaneous import Max, Min
from statannot import add_stat_annotation
import pickle
import os.path
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from matplotlib_venn import venn3
from copy import deepcopy
from cobra.sampling import sample
from scipy import stats
from statsmodels.stats.multitest import multipletests

# Load model

In [252]:
unconstrained_model = cobra.io.read_sbml_model("../mouse_metabolic_model/Khodaee2020SciRep/iMM1865.xml")
unconstrained_model.solver = "cplex"

# Load metabolic genes

In [253]:
USE_FULL_MODEL = True
if USE_FULL_MODEL:
    df_metabolic_gene = pd.read_csv("../data_04082022/metabolic_genes_with_annotation.csv", index_col=0)
    metabolic_genes = set(df_metabolic_gene.index)
else:
    metabolic_genes = set([g.name for g in unconstrained_model.genes])
print('number of metabolic genes = %d'%(len(metabolic_genes)))

number of metabolic genes = 3173


# Load RNASeq data

In [254]:
df_rnaseq1 = pd.read_csv("../data_04082022/2_deseq2_ccr2dep_vs_naive_padj0.05.res.csv", index_col=0)[['ccr2_depleted','naive']]
df_rnaseq2 = pd.read_csv("../data_04082022/3_deseq2_ccr2dep_vs_wt_padj0.05.res.csv", index_col=0)[['wildtype']]
df_rnaseq = pd.merge(df_rnaseq1, df_rnaseq2, left_index=True, right_index=True, how='inner')
df_rnaseq = df_rnaseq.loc[set(df_rnaseq.index).intersection(metabolic_genes)]
df_rnaseq.head()

,ccr2_depleted,naive,wildtype
Fuca2,2291.960451,642.335020,2658.656619
Dad1,329.890056,195.099738,363.801130
Hdac3,199.653155,139.265618,231.927564
Plcl1,4.927242,8.306191,9.095310
Trmt1,65.058988,38.822970,68.189098


## use mean +/- std/2

In [255]:
(np.mean(df_rnaseq['naive'])-np.std(df_rnaseq['naive'])/2, np.mean(df_rnaseq['naive'])+np.std(df_rnaseq['naive'])/2)

(-882.4206276775346, 1792.582287014847)

In [256]:
(np.mean(df_rnaseq['wildtype'])-np.std(df_rnaseq['wildtype'])/2, np.mean(df_rnaseq['wildtype'])+np.std(df_rnaseq['wildtype'])/2)

(-1491.2575359562416, 2605.342801379842)

In [257]:
(np.mean(df_rnaseq['ccr2_depleted'])-np.std(df_rnaseq['ccr2_depleted'])/2, np.mean(df_rnaseq['ccr2_depleted'])+np.std(df_rnaseq['ccr2_depleted'])/2)

(-1690.6848221507603, 2723.7670578999764)

## use 25% and 75% quantile

In [258]:
(np.quantile(df_rnaseq['naive'], 0.25, axis=0), np.quantile(df_rnaseq['naive'], 0.75, axis=0))

(7.291537942000001, 220.0764992)

In [259]:
(np.quantile(df_rnaseq['ccr2_depleted'], 0.25, axis=0), np.quantile(df_rnaseq['ccr2_depleted'], 0.75, axis=0))

(3.8439575039999996, 268.88582210000004)

In [260]:
(np.quantile(df_rnaseq['wildtype'], 0.25, axis=0), np.quantile(df_rnaseq['wildtype'], 0.75, axis=0))

(2.915363666, 278.4090173)

## make sure that when naive is 0, both wildtype and ccr2_depleted have low rnaseq count

In [261]:
df_rnaseq[(df_rnaseq.naive==0) & (df_rnaseq.wildtype>0)].sort_values('wildtype', ascending=False).wildtype.head(5)

Idi1       2.583671
Pnck       2.501887
Pglyrp3    1.275161
Fabp3      1.193377
Pnmt       0.695676
Name: wildtype, dtype: float64

In [262]:
df_rnaseq[(df_rnaseq.naive==0) & (df_rnaseq.ccr2_depleted>0)].sort_values('ccr2_depleted', ascending=False).ccr2_depleted.head(5)

Idi1        6.383801
Fabp3       1.590181
Aqp2        1.590181
Ccnb1ip1    1.557894
Ces3a       1.440416
Name: ccr2_depleted, dtype: float64

# Constraint input fluxes

In [264]:
media = {'EX_glc__D_e'  : -5,          # glucose
         'EX_gln__L_e'  : -0.7,        # glutamine
         'EX_o2_e'      : -1000,       # oxygen
         'EX_pi_e'      : -1000,       # phosphate
         'EX_fe3_e'     : -1000,       # fe3+
         'EX_zn2_e'     : -1000,       # zn2+
         'EX_ca2_e'     : -1000,       # ca2+
         'EX_cl_e'      : -1000,       # cl-
         'EX_k_e'       : -1000,       # k+
         'EX_na1_e'     : -1000,       # na+
         'EX_chol_e'    : -1000,       # choline
         'EX_na1_e'     : -1000,       # sodium
         'EX_hco3_e'    : -1000,       # bicarbonate
         'EX_leu__L_e'  : -0.05,       # leucine
         'EX_ile__L_e'  : -0.05,       # isoleucine
         'EX_pro__L_e'  : -0.05,       # proline
         'EX_trp__L_e'  : -0.05,       # tryptophan
         'EX_met__L_e'  : -0.05,       # methionine
         'EX_cys__L_e'  : -0.05,       # cysteine
         'EX_val__L_e'  : -0.05,       # valine
         'EX_asp__L_e'  : -0.05,       # aspatate
         'EX_asn__L_e'  : -0.05,       # asparagine
         'EX_ser__L_e'  : -0.05,       # serine
         'EX_thr__L_e'  : -0.05,       # threonine
         'EX_ala__L_e'  : -0.05,       # alanine
         'EX_arg__L_e'  : -0.05,       # arginine
         'EX_gly_e'     : -0.05,       # glycine
         'EX_his__L_e'  : -0.05,       # histidine
         'EX_lys__L_e'  : -0.05,       # lysine
         'EX_phe__L_e'  : -0.05,       # phynaalaine
         'EX_glu__L_e'  : -0.05,       # glutamate
         'EX_tyr__L_e'  : -0.05        # tyrosine
        }

ex_rxns = [r for r in unconstrained_model.reactions if r.id.startswith('EX')]
for r in ex_rxns:
    r.lower_bound = 0
    
for r in media.keys():
    try:
        rxn = unconstrained_model.reactions.get_by_id(r)
        rxn.lower_bound = media[r]
        rxn.upper_bound = 1000
    except:
        print("The reaction {} does not exist in model.".format(str(r)))
        continue;

fva = cobra.flux_analysis.variability.flux_variability_analysis(unconstrained_model, unconstrained_model.reactions, loopless=False, fraction_of_optimum=0)
for r in unconstrained_model.reactions:
    r.lower_bound = fva['minimum'][r.id]
    r.upper_bound = fva['maximum'][r.id]

# iMAT

In [265]:
def imat(model, expression_profile, condition, normalization=or2min_and2max, lowq=0.25, highq=0.75, epsilon=0.1):
            
    """
    Parameters
    ----------
    model: genome-scale model
    expression_profile: gene expression profiles
    condition: experimental condition (must be a column in the expression profile)
    normalization: function to process GPR rules if a reaction is associated with multiple isozymes or one enzyme with multiple subunits
    lowq: the quantile to calcualte cutoff value for low expression genes
    highq: the quantile to calculate cutoff value for high expression values
    epsilon: a positive threshold to determine active flux associated with highly expressed reactions
    """
    
    # builds a dict with genes as keys and the expression value for the selected sample.
    gene_exp = dict(zip(expression_profile.index, expression_profile[condition]))
    low_cutoff = np.quantile(expression_profile[condition], lowq, axis=0)
    high_cutoff = np.quantile(expression_profile[condition], highq, axis=0)
    
    # make a disctionary that mappes reaction id to associated gene expression
    gene_id_name_mapping = {g.id:g.name for g in model.genes}
    reaction_profile = {}
    for r in model.reactions:
        # this is an important assumption:
        # we only constrain reactions using gene expression data when all associated genes can be found in expression profiles.
        if len(r.genes) > 0 and all([gene_id_name_mapping[g.id] in list(expression_profile.index) for g in r.genes]):
            _, reaction_profile[r.id] = normalization(r, {g.id: gene_exp[gene_id_name_mapping[g.id]] for g in r.genes})
        
    y_variables = list()
    x_variables = list()
    constraints = list()
    
    # flux variance analysis
    fva_res = cobra.flux_analysis.variability.flux_variability_analysis(model, list(reaction_profile.keys()), loopless=False, fraction_of_optimum=0)
    
    for rid, expression in six.iteritems(reaction_profile):
        if expression == 0.0:
            # reactions shut down completely
            model.reactions.get_by_id(rid).lower_bound = 0
            model.reactions.get_by_id(rid).upper_bound = 0
        elif expression > high_cutoff:     
            reaction = model.reactions.get_by_id(rid)
            y_pos = model.solver.interface.Variable("y_%s_pos" % rid, type="binary")
            y_neg = model.solver.interface.Variable("y_%s_neg" % rid, type="binary")
            y_variables.append([y_neg, y_pos])
            pos_constraint = model.solver.interface.Constraint(
                reaction.flux_expression + y_pos * (fva_res["minimum"][rid] - epsilon),
                lb=fva_res["minimum"][rid], name="pos_highly_%s" % rid)  
            neg_constraint = model.solver.interface.Constraint(
                reaction.flux_expression + y_neg * (fva_res["maximum"][rid] + epsilon),
                ub=fva_res["maximum"][rid], name="neg_highly_%s" % rid)
            constraints.extend([pos_constraint, neg_constraint])
        elif expression < low_cutoff:
            reaction = model.reactions.get_by_id(rid)
            x = model.solver.interface.Variable("x_%s" % rid, type="binary")
            x_variables.append(x)

            pos_constraint = model.solver.interface.Constraint(
                (1 - x) * fva_res["maximum"][rid] - reaction.flux_expression,
                lb=0, name="x_%s_upper" % rid)
            neg_constraint = model.solver.interface.Constraint(
                (1 - x) * fva_res["minimum"][rid] - reaction.flux_expression,
                ub=0, name="x_%s_lower" % rid)
            constraints.extend([pos_constraint, neg_constraint])

    len_xy = 0
    for variable in x_variables:
        model.solver.add(variable)
        len_xy += 1

    for variables in y_variables:
        model.solver.add(variables[0])
        model.solver.add(variables[1])
        len_xy += 1

    for constraint in constraints:
        model.solver.add(constraint)

    model.objective = model.solver.interface.Objective(Add(*[(y[0] + y[1]) for y in y_variables]) + Add(*x_variables), direction="max")
    model.solver.problem.parameters.mip.tolerances.integrality.set(1e-9)
    solution = model.optimize()
    return model, solution, reaction_profile

In [266]:
def or2min_and2max(reaction, gene_values):
    
    """
    Parameters
    ----------
    reaction: reaction to be considered
    gene_values: a dict with gene ids as keys and their expression levels as values
    """
        
    gene_expression_array = reaction.gene_reaction_rule.split(' ')
    gene_expression_array = [x for x in gene_expression_array if x != '']
    if len(gene_expression_array) > 1:        
        gene_expression_array = ['('] + gene_expression_array + [')']
        #print(reaction.id, gene_expression_array)

        # parse substrings from the innermost parenthesis until neither 'or' or 'and' is leftover
        while 'or' in gene_expression_array or 'and' in gene_expression_array:

            # find the inner most ( and )
            for i in range(len(gene_expression_array)):
                if gene_expression_array[i] == ')':
                    # should be the first ')' in the array
                    right_bracket_index = i
                    break
            for i in range(len(gene_expression_array)):
                if gene_expression_array[i] == '(':
                    # should be the last '(' before the first ')'
                    left_bracket_index = i
                if i==right_bracket_index:
                    break

            # parse the content of the innermost parenthesis
            parenthesis_content = gene_expression_array[left_bracket_index+1:right_bracket_index]

            gene_left_behind = ''
            if 'and' in ''.join(parenthesis_content):
                genes = ''.join(parenthesis_content).split('and')
                smallest_value = 1e10
                for g in genes:
                    current_value = gene_values[g]
                    if  current_value < smallest_value:
                        gene_left_behind = g
                        smallest_value = current_value
            elif 'or' in ''.join(parenthesis_content):
                genes = ''.join(parenthesis_content).split('or')
                largest_value = -0.05
                for g in genes:
                    current_value = gene_values[g]
                    if  current_value > largest_value:
                        gene_left_behind = g
                        largest_value = current_value
            else:
                assert len(parenthesis_content)==1
                gene_left_behind = parenthesis_content[0]

            # replace the entire content in the parenthesis with the gene_left_behind
            del gene_expression_array[left_bracket_index:right_bracket_index+1]
            gene_expression_array.insert(left_bracket_index, gene_left_behind)
                
    if len(gene_expression_array)!=1:
        raise ValueError("%s fails assertion test in or2min_and2max."%(reactions.id))
    else:
        return gene_expression_array[0], gene_values[gene_expression_array[0]]

In [268]:
or2min_and2max(unconstrained_model.reactions.get_by_id('24_25VITD2Hm'), {'13081':5})

('13081', 5)

In [269]:
def build_metabolic_model(model, iMAT_solution, expression_profile, condition, normalization=or2min_and2max):
 
    """
    Parameters
    ----------
    model: genome-scale model
    iMAT_solution: iMAT flux solution to constrain reference model
    expression_profile: gene expression profiles
    condition: condition under consideration
    normalization: function to process GPR rules if a reaction is associated with 
                   multiple isozymes or one enzyme with multiple subunits
    """

    gene_exp = dict(zip(expression_profile.index, expression_profile[condition])) # abosolute value
    for r in model.reactions:
        if condition != 'Sensitive':
            if len(r.genes) > 0 and all([identifier.id in expression_profile.index for identifier in r.genes]):
                gid, _ = normalization(r, {g.id: gene_exp.get(g.id) for g in r.genes})
                to_constrain = iMAT_solution[r.id] * expression_profile.loc[gid,condition]
            else:
                to_constrain = iMAT_solution[r.id]
        else:
            to_constrain = iMAT_solution[r.id]
            
        if to_constrain == 0.0:
            r.lower_bound=to_constrain
            r.upper_bound=to_constrain
        elif to_constrain < 0.0:
            r.lower_bound=to_constrain
            
            # remove reversibility
            if r.upper_bound>0.0:
                r.upper_bound=0.0
        else:
            r.upper_bound=to_constrain
            
            # remove reversibility
            if r.lower_bound<0.0:
                r.lower_bound=0.0
        
    return model

# Generate iMAT model for naive cell type

In [270]:
naive_model, naive_solution, _ = imat(
    model=deepcopy(unconstrained_model),
    expression_profile=df_rnaseq,
    condition="naive",
    normalization=or2min_and2max,
    lowq=0.25, 
    highq=0.75, 
    epsilon=0.1)

# Build metabolic models for naive, WT and ccr2-depleted cells

In [271]:
def build_metabolic_model(model, iMAT_solution, expression_profile, condition, normalization=or2min_and2max):
 
    """
    Parameters
    ----------
    model: genome-scale model
    iMAT_solution: iMAT flux solution to constrain reference model
    expression_profile: gene expression profiles
    condition: condition under consideration
    normalization: function to process GPR rules if a reaction is associated with 
                   multiple isozymes or one enzyme with multiple subunits
    """

    gene_exp = dict(zip(expression_profile.index, expression_profile[condition])) # abosolute value
    gene_id_name_mapping = {g.id:g.name for g in model.genes}
    for r in model.reactions:
        if len(r.genes) > 0 and all([gene_id_name_mapping[g.id] in list(expression_profile.index) for g in r.genes]):
            gid, _ = normalization(r, {g.id: gene_exp[gene_id_name_mapping[g.id]] for g in r.genes})
            if expression_profile.loc[gene_id_name_mapping[gid],'naive'] == 0.0:
                # when genes are not expressed in the naive cell, they are generally very low in other conditions
                # so we assume that they follow the same boundary value
                to_constrain = iMAT_solution[r.id]
            else:
                to_constrain = iMAT_solution[r.id] * expression_profile.loc[gene_id_name_mapping[gid],condition] / expression_profile.loc[gene_id_name_mapping[gid],'naive']
        else:
            to_constrain = iMAT_solution[r.id]
            
        if to_constrain == 0.0:
            r.lower_bound=to_constrain
            r.upper_bound=to_constrain
        elif to_constrain < 0.0:
            r.lower_bound=to_constrain
            if r.upper_bound>0.0:
                r.upper_bound=0.0
        else:
            r.upper_bound=to_constrain
            if r.lower_bound<0.0:
                r.lower_bound=0.0
        
    return model

In [272]:
constrained_models = {} # models constrained by gene expression data
flux_samples = {} # sampling flux solution space of constrained models

conditions=['naive','wildtype','ccr2_depleted']
for cond in conditions:
         
    # construct model
    print('building model for condition: %s'%(cond))
    model = build_metabolic_model(model=deepcopy(unconstrained_model), 
                                  iMAT_solution=naive_solution,
                                  expression_profile=df_rnaseq,
                                  condition=cond)
    constrained_models.update({cond: model})
    
    # sample the flux space
    print('sampling solution space for condition-specific model (condition = %s)'%(cond))
    s = sample(model, 10000, processes=12)
    flux_samples.update({cond : s})

# consider save the variables to file so that you can load them next time without running from scratch
# write variables:
pickle.dump(constrained_models, open("constrained_models.p", "wb"))
pickle.dump(flux_samples, open("flux_samples.p", "wb"))

building model for condition: naive
sampling solution space for condition-specific model (condition = naive)
building model for condition: wildtype
sampling solution space for condition-specific model (condition = wildtype)
building model for condition: ccr2_depleted
sampling solution space for condition-specific model (condition = ccr2_depleted)


# Identify fluxes significant different between each other

In [273]:
# load variables:
constrained_models = pickle.load(open("constrained_models.p","rb"))
flux_samples = pickle.load(open("flux_samples.p","rb"))

In [285]:
paired_conditions = [('naive','wildtype'),('naive','ccr2_depleted'),('wildtype','ccr2_depleted')]
flux_res = []
for pair in paired_conditions:
    cond1 = pair[0]
    cond2 = pair[1]
    flux_samples1 = flux_samples[cond1]
    flux_samples2 = flux_samples[cond2]
    for rid in flux_samples1.columns:
        x = list(flux_samples1[rid])
        y = list(flux_samples2[rid])
        if np.max(np.abs(x))==0 and np.max(np.abs(y))==0:
            continue
        else:
            log2_mfr = np.log2(np.mean(x)/np.mean(y))
            pvalue = stats.ks_2samp(x, y)[1]
            flux_res.append([cond1, cond2, rid, np.mean(x), np.mean(y), log2_mfr, pvalue])
df_flux_res = pd.DataFrame(flux_res, columns=['Condition1','Condition2','Reaction','MeanFluxCond1','MeanFluxCond2','Log2MeanFluxRatio','P'])
df_flux_res['Padj'] = multipletests(df_flux_res.P, alpha=0.05, method='fdr_bh')[1]
df_flux_res.to_csv("flux_sampling_comparison_summary.csv", index=False)
df_flux_res.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log2
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log2
  


,Condition1,Condition2,Reaction,MeanFluxCond1,MeanFluxCond2,Log2MeanFluxRatio,P,Padj
0,naive,wildtype,10FTHF7GLUtl,0.047169,0.050554,-0.099989,1.674592e-47,1.776560e-47
1,naive,wildtype,10FTHFtl,-0.047169,-0.050554,-0.099989,1.674592e-47,1.776560e-47
2,naive,wildtype,10FTHFtm,467.427271,108.454557,2.107651,0.000000e+00,0.000000e+00
3,naive,wildtype,2HBO,-0.053233,-0.025287,1.073907,0.000000e+00,0.000000e+00
4,naive,wildtype,2HBt2,-0.062086,-0.031279,0.989073,0.000000e+00,0.000000e+00


In [291]:
df_flux_res[(df_flux_res.Condition1=='naive') & (df_flux_res.Condition2=='wildtype')].sort_values('Log2MeanFluxRatio', ascending=False).head(30)

,Condition1,Condition2,Reaction,MeanFluxCond1,MeanFluxCond2,Log2MeanFluxRatio,P,Padj
3291,naive,wildtype,GD1Atg,0.055082,0.0,inf,0.0,0.0
1754,naive,wildtype,RE2994X,0.054533,0.0,inf,0.0,0.0
3865,naive,wildtype,HMR_3346,0.054520,0.0,inf,0.0,0.0
3864,naive,wildtype,HMR_3345,0.054520,0.0,inf,0.0,0.0
3863,naive,wildtype,HMR_3344,0.054520,0.0,inf,0.0,0.0
3461,naive,wildtype,HMR_0823,0.051022,0.0,inf,0.0,0.0
3862,naive,wildtype,HMR_3343,0.054520,0.0,inf,0.0,0.0
3468,naive,wildtype,HMR_0837,0.053706,0.0,inf,0.0,0.0
3861,naive,wildtype,HMR_3342,0.054520,0.0,inf,0.0,0.0
1756,naive,wildtype,RE2997X,0.054533,0.0,inf,0.0,0.0


In [296]:
unconstrained_model.reactions.EX_h2o2_e

Reaction identifier,EX_h2o2_e
Name,Hydrogen peroxide exchange
Memory address,0x01399c2590
Stoichiometry,h2o2_e --> Hydrogen Peroxide -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [298]:
df_flux_res[df_flux_res.Reaction=="EX_h2o2_e"]

,Condition1,Condition2,Reaction,MeanFluxCond1,MeanFluxCond2,Log2MeanFluxRatio,P,Padj


In [303]:
flux_samples['naive']["EX_h2o2_e"].mean()

0.0